# Ensemble (voting)

In [1]:
from si.data import Dataset
from si.util import summary#, CrossValidationScore
import os

In [2]:
DIR = os.path.dirname(os.path.realpath('.'))
filename = os.path.join(DIR, 'datasets/breast-bin.data')
dataset = Dataset.from_data(filename)
summary(dataset)

,mean,std,var,max,min
A,4.417740,2.813726,7.917053,10.0,1.0
B,3.134478,3.049276,9.298082,10.0,1.0
C,3.207439,2.969786,8.819630,10.0,1.0
D,2.806867,2.853336,8.141527,10.0,1.0
E,3.216023,2.212715,4.896110,10.0,1.0
F,3.486409,3.619337,13.099601,10.0,1.0
G,3.437768,2.436619,5.937114,10.0,1.0
H,2.866953,3.051449,9.311340,10.0,1.0
I,1.589413,1.713851,2.937284,10.0,1.0
y,0.344778,0.475296,0.225906,1.0,0.0


In [3]:
dataset.toDataframe()

,A,B,C,D,E,F,G,H,I,y
0,8.0,4.0,4.0,1.0,2.0,9.0,3.0,3.0,1.0,1.0
1,5.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,0.0
2,2.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,0.0
3,2.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,0.0
4,9.0,5.0,8.0,1.0,2.0,3.0,2.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...
694,1.0,1.0,1.0,1.0,4.0,3.0,1.0,1.0,1.0,0.0
695,5.0,3.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,0.0
696,5.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,0.0
697,2.0,3.0,2.0,2.0,2.0,2.0,3.0,1.0,1.0,0.0


In [4]:
# Use accuracy as scorring function
from si.util.metrics import accuracy

### Decision Tree

In [5]:
from si.supervised import DecisionTree

In [6]:
dt = DecisionTree()

In [7]:
cv = CrossValidationScore(dt,dataset,score=accuracy)
cv.run()
cv.toDataframe()

NameError: name 'CrossValidationScore' is not defined

### Logistic regression

In [ ]:
from si.supervised import LogisticRegression
logreg = LogisticRegression()


In [ ]:
cv = CrossValidationScore(logreg,dataset,score=accuracy)
cv.run()
cv.toDataframe()

### KNN

In [ ]:
from si.supervised import KNN

In [ ]:
knn = KNN(7)

In [ ]:
cv = CrossValidationScore(knn,dataset,score=accuracy)
cv.run()
cv.toDataframe()

## Ensemble

In [ ]:
from si.supervised import Essemble,majority
en = Essemble([dt,logreg,knn], majority, accuracy())

In [ ]:
cv = CrossValidationScore(en,dataset,score=accuracy())
cv.run()
cv.toDataframe()